In [1]:
import os
os.chdir(r"C:\Users\user\DACON Dropbox\1. 대회\13th_제주테크노파크2\데이터 정제\1. 데이터\2. 최종 데이터")

import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
train.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,...,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,...,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,...,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.0,17.0,...,26.0,14.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0
4,4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.41260,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


# 1. date 변수 변환

date 컬럼을 판다스의 datetime 변수형으로 변환한다.

In [4]:
train['date'] = pd.to_datetime(train['date'])

date 컬럼으로부터 요일을 뽑아낸다.

In [5]:
train['weekday'] = train['date'].dt.weekday

파이썬 도큐멘테이션에는 weekday를 아래와 같이 설명하고 있다.

Return the day of the week as an integer, where Monday is 0 and Sunday is 6.

weekday 변수 생성 후 pandas의 get_dummies 함수를 활용하여 weekday에 대한 one hot encoding을 실행한다.

In [6]:
train = pd.get_dummies(train,columns=['weekday'])

test 데이터에 대해서도 위와 똑같은 변환을 해준다.

In [7]:
test['date'] = pd.to_datetime(test['date'])
test['weekday'] = test['date'].dt.weekday
test = pd.get_dummies(test,columns=['weekday'])

# 2. in_out 변수 변환

In [8]:
train['in_out'].value_counts()

시내    408500
시외      6923
Name: in_out, dtype: int64

보다시피 '시외'인 것이 매우 적다.

'시외'를 1로, '시내'를 0으로 변환한다.

In [9]:
train['in_out'] = train['in_out'].map({'시내':0,'시외':1})

test 데이터에 대해서도 위와 똑같은 변환을 해준다.

In [10]:
test['in_out'] = test['in_out'].map({'시내':0,'시외':1})

# 3. 좌표 데이터를 이용한 Feature Engineering

제주도의 인구는 서귀포시와 제주시에 몰려 있는 경우가 많다. 그러므로 해당 지역과의 거리를 각각 feature로 추가하겠다.

<strong>dis_jejusi</strong> : 버스정류장과 제주시와의 거리


<strong>dis_seoquipo</strong> : 버스정류장과 서귀포시와의 거리

<strong>제주시의 위.경도</strong> : 33.500770, 126.522761

<strong>서귀포시의 위.경도</strong> : 33.259429, 126.558217

2 개의 좌표 간의 거리를 구하는 함수는 파이썬의 geopy 패키지에 존재한다.

In [11]:
# !pip install geopy

In [12]:
import geopy.distance

coords_jejusi = (33.500770, 126.522761) #제주시의 위도 경도
coords_seoquipo = (33.259429, 126.558217) #서귀포시의 위도 경도


train['dis_jejusi'] = [geopy.distance.vincenty((train['latitude'].iloc[i],train['longitude'].iloc[i]), coords_jejusi).km for i in range(len(train))]
train['dis_seoquipo'] = [geopy.distance.vincenty((train['latitude'].iloc[i],train['longitude'].iloc[i]), coords_seoquipo).km for i in range(len(train))]

In [13]:
train['dis_jejusi'].corr(train['18~20_ride'])

-0.12380548460763087

In [14]:
train['dis_seoquipo'].corr(train['18~20_ride'])

0.022087013666357798

제주시와의 거리가 멀면 멀수록 승차인원이 줄어드는 경향을 보이나 서귀포시와의 거리는 모호한 상관성을 띄고 있다.

test 데이터에 대해서도 위와 똑같은 변환을 해준다.

In [15]:
test['dis_jejusi'] = [geopy.distance.vincenty((test['latitude'].iloc[i],test['longitude'].iloc[i]), coords_jejusi).km for i in range(len(test))]
test['dis_seoquipo'] = [geopy.distance.vincenty((test['latitude'].iloc[i],test['longitude'].iloc[i]), coords_seoquipo).km for i in range(len(test))]

# 4. randomforest을 활용한 모델링

sklearn 내부의 randomforest 함수를 이용해 모델링을 진행한다.

머신러닝 학습에 활용할 input 변수와 target 변수를 정의한다.

In [16]:
input_var=['in_out','latitude','longitude','6~7_ride', '7~8_ride', '8~9_ride', '9~10_ride',
       '10~11_ride', '11~12_ride', '6~7_takeoff', '7~8_takeoff', '8~9_takeoff',
       '9~10_takeoff', '10~11_takeoff', '11~12_takeoff','weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4',
       'weekday_5', 'weekday_6', 'dis_jejusi', 'dis_seoquipo']
target=['18~20_ride']

In [17]:
X_train=train[input_var]
y_train=train[target]

X_test=test[input_var]

랜덤포레스트 모델을 정의한다.

In [18]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=1217)

모델을 학습한다.

In [19]:
rf.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=1217,
                      verbose=0, warm_start=False)

학습한 모델을 이용해 예측을 진행하고 이를 submission.csv라는 파일로 저장한다.

In [20]:
test['18~20_ride'] = rf.predict(X_test)
test[['id','18~20_ride']].to_csv("dacon_base_line.csv",index=False)